# Cleaning of the road inventory

## The first part of this notebook uses the road inventory dictionary to constrain the values of data from the shapefile.

In [1]:
# First, import and load in data.

import inventoryCleaner as iC
import pandas as pd
import numpy as np
import csv
import string
%matplotlib inline

fpath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RoadInventory.geojson'
pdinv = iC.PDInventory('roadInventory', 'test')

pdinv.geojsontodf(fpath)




In [10]:
# Combine the two street name variables and remove the old ones
nu = pd.isnull(pdinv.table.streetname)
nu2 = pd.isnull(pdinv.table.streetna_1)

pdinv.table.streetname[nu]=''
pdinv.table.streetna_1[nu2]=''

pdinv.table["newstreetn"] = pdinv.table["streetname"].map(str) + pdinv.table["streetna_1"]

pdinv.table.drop('streetname', axis=1, inplace=True)
pdinv.table.drop('streetna_1', axis=1, inplace=True)

0

In [17]:
pdinv.table.columns

Index(['accesscont', 'addedroadt', 'adt', 'adtderivat', 'adtstation',
       'adtyear', 'assigned_1', 'assignedle', 'city', 'county', 'crn', 'curbs',
       'dateactive', 'equation_1', 'equationro', 'facilityty', 'federalaid',
       'federalfun', 'fromcity', 'fromendtyp', 'frommeasur', 'fromstate',
       'fromstreet', 'functional', 'futurefaci', 'hpmscode', 'hpmssample',
       'iri', 'iristatus', 'iriyear', 'item_id', 'jurisdicti', 'leftshou_1',
       'leftshould', 'leftsidewa', 'lifecycles', 'massdothig', 'mediantype',
       'medianwidt', 'mhs', 'mileagecou', 'mpo', 'municipals', 'nhsstatus',
       'numberofpe', 'numberoftr', 'opposingdi', 'oppositenu', 'psi',
       'psiyear', 'rightofway', 'rightsho_1', 'rightshoul', 'rightsidew',
       'roadinvent', 'roadsegmen', 'routedirec', 'routefrom', 'routekey',
       'routenumbe', 'routequali', 'routesyste', 'routeto', 'routetype', 'rpa',
       'shape_len', 'speedlimit', 'streetlist', 'streetoper', 'structural',
       'subroute', '

In [2]:
# This cell loads in a table of requrements I collated.
pathtoreqs = '/home/louisf/Documents/Insight/massdriver/docs/road_inventory_params.csv'
reqs = pd.read_csv(pathtoreqs, delimiter = '^')

In [3]:
# I now go over each variable in the table and summarize it. 

coldesc = {}

for col in pdinv.table.columns:
    coldesc[col] = {'type':str(pdinv.table.dtypes[col]), 
                    'uniques':pdinv.table[col].unique()
                   }

In [50]:
it = 0
for row in reqs.iterrows():
    if it == 0:
        it += 1
        continue
    a = row[1]
    varname = a['Variable']
    vartype = a['Data type']
    t = pdinv.table[varname]
    if a['Required'] == 'yes':
        vals = pdinv.table[varname]
        print('variable : ' + varname + ', null values: ' + str(sum(pd.isnull(vals))))
        if pd.notnull(a['Reasonable range']):
            s = a['Reasonable range'].split('-')
            if s[0] in string.ascii_letters:
                rlower = s[0]
                rupper = s[1]
                r = string.ascii_letters
            else:
                rlower = int(s[0])
                rupper = int(s[1])
                r = range(rlower, rupper + 1)
            
            print('All values within reasonable range: ' + str(vals.all() in r))
    it+=1
    
    
    
    
# Based on my analysis so far, I should be dropping routesyste and the old streetname variables from consideration.

variable : assigned_1, null values: 0
All values within reasonable range: True
variable : assignedle, null values: 0
variable : city, null values: 0
All values within reasonable range: True
variable : county, null values: 0
All values within reasonable range: True
variable : crn, null values: 0
variable : dateactive, null values: 0
variable : fromcity, null values: 0
All values within reasonable range: False
variable : fromendtyp, null values: 0
All values within reasonable range: False
variable : frommeasur, null values: 0
variable : item_id, null values: 0
variable : leftshou_1, null values: 0
variable : lifecycles, null values: 0
All values within reasonable range: True
variable : massdothig, null values: 0
All values within reasonable range: True
variable : mileagecou, null values: 0
All values within reasonable range: True
variable : mpo, null values: 0
variable : municipals, null values: 0
All values within reasonable range: True
variable : roadinvent, null values: 0
variable : r

In [62]:
np.max(pdinv.table['fromendtyp'])

6

In [45]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [28]:
s = a['Reasonable range'].split('-')
rlower = int(s[0])
print(rlower)
rupper = int(s[1])
print(rupper)
print(range(rlower, rupper+1))

0
2
range(0, 3)
